# [Advent of Code 2018](https://adventofcode.com/2018)

A collection of common functions to be used across problems, will add more as a generic use case for each comes up!

In [1]:
def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2018/input{}.txt'.format(day))
    return open(filename)

## Day 1